### In our last model in file "Music_Generation_Train1.ipynb", we got 83.6% accuracy. However, in order to generate melodious music, we need at least 90% accuracy. 
### So, we have loaded the weights of last epoch from our previous model into this model and also we have added 2 extra layers of LSTM here with more LSTM units. 
### Here, we are fine-tuning our old layers and we have added more layers. In short, here we are doing "Transfer Learning" from old to new model.

In [1]:
import os
import json
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dropout, TimeDistributed, Dense, Activation, Embedding

Using TensorFlow backend.


In [2]:
data_directory = "../Data2/"
data_file = "Data_Tunes.txt"
charIndex_json = "char_to_index.json"
model_weights_directory = '../Data2/Model_Weights/'
BATCH_SIZE = 16
SEQ_LENGTH = 64

In [3]:
def read_batches(all_chars, unique_chars):
    length = all_chars.shape[0]
    batch_chars = int(length / BATCH_SIZE) #155222/16 = 9701
    
    for start in range(0, batch_chars - SEQ_LENGTH, 64):  #(0, 9637, 64)  #it denotes number of batches. It runs everytime when
        #new batch is created. We have a total of 151 batches.
        X = np.zeros((BATCH_SIZE, SEQ_LENGTH))    #(16, 64)
        Y = np.zeros((BATCH_SIZE, SEQ_LENGTH, unique_chars))   #(16, 64, 87)
        for batch_index in range(0, 16):  #it denotes each row in a batch.  
            for i in range(0, 64):  #it denotes each column in a batch. Each column represents each character means 
                #each time-step character in a sequence.
                X[batch_index, i] = all_chars[batch_index * batch_chars + start + i]
                Y[batch_index, i, all_chars[batch_index * batch_chars + start + i + 1]] = 1 #here we have added '1' because the
                #correct label will be the next character in the sequence. So, the next character will be denoted by
                #all_chars[batch_index * batch_chars + start + i] + 1. 
        yield X, Y

In [7]:
def built_model(batch_size, seq_length, unique_chars):
    model = Sequential()
    
    model.add(Embedding(input_dim = unique_chars, output_dim = 512, batch_input_shape = (batch_size, seq_length), name = "embd_1")) 
    
    model.add(LSTM(512, return_sequences = True, stateful = True, name = "lstm_first"))
    model.add(Dropout(0.2, name = "drp_1"))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(LSTM(256, return_sequences = True, stateful = True))
    model.add(Dropout(0.2))
    
    model.add(TimeDistributed(Dense(unique_chars)))
    model.add(Activation("softmax"))
    
    model.load_weights("../Data/Model_Weights/Weights_80.h5", by_name = True)
    
    return model

In [8]:
def training_model(data, epochs = 90):
    #mapping character to index
    char_to_index = {ch: i for (i, ch) in enumerate(sorted(list(set(data))))}
    print("Number of unique characters in our whole tunes database = {}".format(len(char_to_index))) #87
    
    with open(os.path.join(data_directory, charIndex_json), mode = "w") as f:
        json.dump(char_to_index, f)
        
    index_to_char = {i: ch for (ch, i) in char_to_index.items()}
    unique_chars = len(char_to_index)
    
    model = built_model(BATCH_SIZE, SEQ_LENGTH, unique_chars)
    model.summary()
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    
    all_characters = np.asarray([char_to_index[c] for c in data], dtype = np.int32)
    print("Total number of characters = "+str(all_characters.shape[0])) #155222
    
    epoch_number, loss, accuracy = [], [], []
    
    for epoch in range(epochs):
        print("Epoch {}/{}".format(epoch+1, epochs))
        final_epoch_loss, final_epoch_accuracy = 0, 0
        epoch_number.append(epoch+1)
        
        for i, (x, y) in enumerate(read_batches(all_characters, unique_chars)):
            final_epoch_loss, final_epoch_accuracy = model.train_on_batch(x, y) 
            print("Batch: {}, Loss: {}, Accuracy: {}".format(i+1, final_epoch_loss, final_epoch_accuracy))
            #here, above we are reading the batches one-by-one and train our model on each batch one-by-one.
        loss.append(final_epoch_loss)
        accuracy.append(final_epoch_accuracy)
        
        #saving weights after every 10 epochs
        if (epoch + 1) % 10 == 0:
            if not os.path.exists(model_weights_directory):
                os.makedirs(model_weights_directory)
            model.save_weights(os.path.join(model_weights_directory, "Weights_{}.h5".format(epoch+1)))
            print('Saved Weights at epoch {} to file Weights_{}.h5'.format(epoch+1, epoch+1))
    
    #creating dataframe and record all the losses and accuracies at each epoch
    log_frame = pd.DataFrame(columns = ["Epoch", "Loss", "Accuracy"])
    log_frame["Epoch"] = epoch_number
    log_frame["Loss"] = loss
    log_frame["Accuracy"] = accuracy
    log_frame.to_csv("../Data2/log.csv", index = False)

In [9]:
file = open(os.path.join(data_directory, data_file), mode = 'r')
data = file.read()
file.close()
if __name__ == "__main__":
    training_model(data)

Number of unique characters in our whole tunes database = 87
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embd_1 (Embedding)           (16, 64, 512)             44544     
_________________________________________________________________
lstm_first (LSTM)            (16, 64, 512)             2099200   
_________________________________________________________________
drp_1 (Dropout)              (16, 64, 512)             0         
_________________________________________________________________
lstm_3 (LSTM)                (16, 64, 256)             787456    
_________________________________________________________________
dropout_3 (Dropout)          (16, 64, 256)             0         
_________________________________________________________________
lstm_4 (LSTM)                (16, 64, 256)             525312    
_________________________________________________________________
dropout_4 (Drop

Batch: 104, Loss: 2.6799540519714355, Accuracy: 0.2822265625
Batch: 105, Loss: 2.6733603477478027, Accuracy: 0.2734375
Batch: 106, Loss: 2.7433605194091797, Accuracy: 0.27734375
Batch: 107, Loss: 2.7803614139556885, Accuracy: 0.26953125
Batch: 108, Loss: 2.737197160720825, Accuracy: 0.271484375
Batch: 109, Loss: 2.6563286781311035, Accuracy: 0.2900390625
Batch: 110, Loss: 2.5465445518493652, Accuracy: 0.2705078125
Batch: 111, Loss: 2.5659360885620117, Accuracy: 0.298828125
Batch: 112, Loss: 2.6764955520629883, Accuracy: 0.287109375
Batch: 113, Loss: 2.7359070777893066, Accuracy: 0.2783203125
Batch: 114, Loss: 2.5767500400543213, Accuracy: 0.3203125
Batch: 115, Loss: 2.651116371154785, Accuracy: 0.30859375
Batch: 116, Loss: 2.6577837467193604, Accuracy: 0.2958984375
Batch: 117, Loss: 2.6628894805908203, Accuracy: 0.2861328125
Batch: 118, Loss: 2.6720306873321533, Accuracy: 0.28515625
Batch: 119, Loss: 2.7785773277282715, Accuracy: 0.265625
Batch: 120, Loss: 2.584747791290283, Accuracy: 

Batch: 93, Loss: 1.8988145589828491, Accuracy: 0.453125
Batch: 94, Loss: 1.8799166679382324, Accuracy: 0.46484375
Batch: 95, Loss: 1.8632824420928955, Accuracy: 0.4638671875
Batch: 96, Loss: 1.9557894468307495, Accuracy: 0.4541015625
Batch: 97, Loss: 1.8819596767425537, Accuracy: 0.4892578125
Batch: 98, Loss: 1.8350690603256226, Accuracy: 0.51171875
Batch: 99, Loss: 1.7719873189926147, Accuracy: 0.5009765625
Batch: 100, Loss: 1.7834796905517578, Accuracy: 0.494140625
Batch: 101, Loss: 1.8639140129089355, Accuracy: 0.4931640625
Batch: 102, Loss: 1.7918750047683716, Accuracy: 0.482421875
Batch: 103, Loss: 2.001720666885376, Accuracy: 0.466796875
Batch: 104, Loss: 1.7587594985961914, Accuracy: 0.4912109375
Batch: 105, Loss: 1.8558669090270996, Accuracy: 0.4609375
Batch: 106, Loss: 1.9183728694915771, Accuracy: 0.453125
Batch: 107, Loss: 2.0028438568115234, Accuracy: 0.451171875
Batch: 108, Loss: 2.015927791595459, Accuracy: 0.453125
Batch: 109, Loss: 1.9852508306503296, Accuracy: 0.434570

Batch: 81, Loss: 1.6082406044006348, Accuracy: 0.4921875
Batch: 82, Loss: 1.6187214851379395, Accuracy: 0.5
Batch: 83, Loss: 1.4849236011505127, Accuracy: 0.5595703125
Batch: 84, Loss: 1.5858116149902344, Accuracy: 0.5458984375
Batch: 85, Loss: 1.4814954996109009, Accuracy: 0.568359375
Batch: 86, Loss: 1.7775059938430786, Accuracy: 0.4765625
Batch: 87, Loss: 1.5642951726913452, Accuracy: 0.5458984375
Batch: 88, Loss: 1.6903676986694336, Accuracy: 0.521484375
Batch: 89, Loss: 1.7112826108932495, Accuracy: 0.4892578125
Batch: 90, Loss: 1.5595171451568604, Accuracy: 0.5380859375
Batch: 91, Loss: 1.578244924545288, Accuracy: 0.53125
Batch: 92, Loss: 1.6442663669586182, Accuracy: 0.4990234375
Batch: 93, Loss: 1.518354058265686, Accuracy: 0.5458984375
Batch: 94, Loss: 1.5242104530334473, Accuracy: 0.5361328125
Batch: 95, Loss: 1.5516424179077148, Accuracy: 0.5224609375
Batch: 96, Loss: 1.5946635007858276, Accuracy: 0.5361328125
Batch: 97, Loss: 1.4991743564605713, Accuracy: 0.564453125
Batch

Batch: 70, Loss: 1.498258352279663, Accuracy: 0.5478515625
Batch: 71, Loss: 1.478144884109497, Accuracy: 0.5419921875
Batch: 72, Loss: 1.3667806386947632, Accuracy: 0.5810546875
Batch: 73, Loss: 1.437915563583374, Accuracy: 0.5751953125
Batch: 74, Loss: 1.3606884479522705, Accuracy: 0.568359375
Batch: 75, Loss: 1.3232873678207397, Accuracy: 0.5869140625
Batch: 76, Loss: 1.4881353378295898, Accuracy: 0.5068359375
Batch: 77, Loss: 1.4730709791183472, Accuracy: 0.5380859375
Batch: 78, Loss: 1.4198012351989746, Accuracy: 0.57421875
Batch: 79, Loss: 1.281046986579895, Accuracy: 0.6298828125
Batch: 80, Loss: 1.3164995908737183, Accuracy: 0.578125
Batch: 81, Loss: 1.4972001314163208, Accuracy: 0.50390625
Batch: 82, Loss: 1.453352928161621, Accuracy: 0.51953125
Batch: 83, Loss: 1.3095250129699707, Accuracy: 0.5986328125
Batch: 84, Loss: 1.3743250370025635, Accuracy: 0.58203125
Batch: 85, Loss: 1.3066214323043823, Accuracy: 0.5908203125
Batch: 86, Loss: 1.575434923171997, Accuracy: 0.5185546875

...

Batch: 73, Loss: 0.20262694358825684, Accuracy: 0.93359375
Batch: 74, Loss: 0.1947670876979828, Accuracy: 0.9375
Batch: 75, Loss: 0.1415400505065918, Accuracy: 0.9560546875
Batch: 76, Loss: 0.15575379133224487, Accuracy: 0.9453125
Batch: 77, Loss: 0.13386106491088867, Accuracy: 0.95703125
Batch: 78, Loss: 0.19934695959091187, Accuracy: 0.9384765625
Batch: 79, Loss: 0.20579111576080322, Accuracy: 0.9306640625
Batch: 80, Loss: 0.12959280610084534, Accuracy: 0.9521484375
Batch: 81, Loss: 0.17432203888893127, Accuracy: 0.9453125
Batch: 82, Loss: 0.19839343428611755, Accuracy: 0.9375
Batch: 83, Loss: 0.15744179487228394, Accuracy: 0.94140625
Batch: 84, Loss: 0.22427721321582794, Accuracy: 0.9267578125
Batch: 85, Loss: 0.1965397447347641, Accuracy: 0.939453125
Batch: 86, Loss: 0.1898457407951355, Accuracy: 0.93359375
Batch: 87, Loss: 0.17481212317943573, Accuracy: 0.94140625
Batch: 88, Loss: 0.18817532062530518, Accuracy: 0.9375
Batch: 89, Loss: 0.16862347722053528, Accuracy: 0.94921875
Batc

Batch: 59, Loss: 0.19486531615257263, Accuracy: 0.93359375
Batch: 60, Loss: 0.1609058380126953, Accuracy: 0.94140625
Batch: 61, Loss: 0.14823311567306519, Accuracy: 0.9580078125
Batch: 62, Loss: 0.14636683464050293, Accuracy: 0.9521484375
Batch: 63, Loss: 0.1538703441619873, Accuracy: 0.9482421875
Batch: 64, Loss: 0.17371302843093872, Accuracy: 0.943359375
Batch: 65, Loss: 0.1694931983947754, Accuracy: 0.94140625
Batch: 66, Loss: 0.20049485564231873, Accuracy: 0.9365234375
Batch: 67, Loss: 0.15999990701675415, Accuracy: 0.9521484375
Batch: 68, Loss: 0.18521565198898315, Accuracy: 0.9462890625
Batch: 69, Loss: 0.1984749138355255, Accuracy: 0.9296875
Batch: 70, Loss: 0.1575949788093567, Accuracy: 0.951171875
Batch: 71, Loss: 0.17616356909275055, Accuracy: 0.935546875
Batch: 72, Loss: 0.15032804012298584, Accuracy: 0.953125
Batch: 73, Loss: 0.16633227467536926, Accuracy: 0.943359375
Batch: 74, Loss: 0.20307350158691406, Accuracy: 0.923828125
Batch: 75, Loss: 0.14941845834255219, Accuracy:

Batch: 45, Loss: 0.1788504272699356, Accuracy: 0.94921875
Batch: 46, Loss: 0.18431277573108673, Accuracy: 0.9443359375
Batch: 47, Loss: 0.21798855066299438, Accuracy: 0.9228515625
Batch: 48, Loss: 0.16608940064907074, Accuracy: 0.9482421875
Batch: 49, Loss: 0.1842653751373291, Accuracy: 0.9375
Batch: 50, Loss: 0.1731337010860443, Accuracy: 0.9375
Batch: 51, Loss: 0.17133116722106934, Accuracy: 0.943359375
Batch: 52, Loss: 0.16645388305187225, Accuracy: 0.9443359375
Batch: 53, Loss: 0.1666642725467682, Accuracy: 0.9423828125
Batch: 54, Loss: 0.16889825463294983, Accuracy: 0.94921875
Batch: 55, Loss: 0.19643007218837738, Accuracy: 0.943359375
Batch: 56, Loss: 0.21262997388839722, Accuracy: 0.931640625
Batch: 57, Loss: 0.2003868818283081, Accuracy: 0.939453125
Batch: 58, Loss: 0.18847888708114624, Accuracy: 0.94140625
Batch: 59, Loss: 0.19573603570461273, Accuracy: 0.9384765625
Batch: 60, Loss: 0.14207494258880615, Accuracy: 0.9521484375
Batch: 61, Loss: 0.17160549759864807, Accuracy: 0.9

Batch: 31, Loss: 0.15163184702396393, Accuracy: 0.953125
Batch: 32, Loss: 0.16688258945941925, Accuracy: 0.9443359375
Batch: 33, Loss: 0.18729472160339355, Accuracy: 0.9375
Batch: 34, Loss: 0.19577939808368683, Accuracy: 0.93359375
Batch: 35, Loss: 0.17797794938087463, Accuracy: 0.935546875
Batch: 36, Loss: 0.21053817868232727, Accuracy: 0.9384765625
Batch: 37, Loss: 0.19394078850746155, Accuracy: 0.9306640625
Batch: 38, Loss: 0.16510087251663208, Accuracy: 0.9482421875
Batch: 39, Loss: 0.17164947092533112, Accuracy: 0.9423828125
Batch: 40, Loss: 0.1796838939189911, Accuracy: 0.947265625
Batch: 41, Loss: 0.15093602240085602, Accuracy: 0.9521484375
Batch: 42, Loss: 0.16322192549705505, Accuracy: 0.9541015625
Batch: 43, Loss: 0.15728378295898438, Accuracy: 0.955078125
Batch: 44, Loss: 0.1795010268688202, Accuracy: 0.9345703125
Batch: 45, Loss: 0.15367287397384644, Accuracy: 0.9443359375
Batch: 46, Loss: 0.17340096831321716, Accuracy: 0.9384765625
Batch: 47, Loss: 0.19971513748168945, Acc

Batch: 17, Loss: 0.1684514433145523, Accuracy: 0.9453125
Batch: 18, Loss: 0.15818026661872864, Accuracy: 0.94921875
Batch: 19, Loss: 0.20756500959396362, Accuracy: 0.935546875
Batch: 20, Loss: 0.17832988500595093, Accuracy: 0.943359375
Batch: 21, Loss: 0.1430189609527588, Accuracy: 0.9462890625
Batch: 22, Loss: 0.18700286746025085, Accuracy: 0.9404296875
Batch: 23, Loss: 0.18987783789634705, Accuracy: 0.939453125
Batch: 24, Loss: 0.1886029839515686, Accuracy: 0.9384765625
Batch: 25, Loss: 0.15327878296375275, Accuracy: 0.9501953125
Batch: 26, Loss: 0.13169878721237183, Accuracy: 0.95703125
Batch: 27, Loss: 0.15969370305538177, Accuracy: 0.9501953125
Batch: 28, Loss: 0.15482723712921143, Accuracy: 0.94921875
Batch: 29, Loss: 0.16475284099578857, Accuracy: 0.9423828125
Batch: 30, Loss: 0.15739938616752625, Accuracy: 0.9521484375
Batch: 31, Loss: 0.18169370293617249, Accuracy: 0.931640625
Batch: 32, Loss: 0.17363029718399048, Accuracy: 0.9443359375
Batch: 33, Loss: 0.18652798235416412, Ac

In [10]:
log = pd.read_csv(os.path.join(data_directory, "log.csv"))
log

,Epoch,Loss,Accuracy
0,1,2.335064,0.348633
1,2,1.779615,0.505859
2,3,1.450759,0.568359
3,4,1.333034,0.587891
4,5,1.227172,0.613281
5,6,1.146765,0.640625
6,7,1.068224,0.657227
7,8,1.020259,0.675781
8,9,0.967078,0.688477
9,10,0.907838,0.720703
